In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import category_encoders as ce
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [ ]:
#bring in train data
train_df = pd.read_csv('../input/train.csv')
#bring in test data
test_df = pd.read_csv('../input/test.csv')

In [ ]:
#check the numbers of samples and features
print("The train data size before dropping Id feature is : {} ".format(train_df.shape))
print("The test data size before dropping Id feature is : {} ".format(test_df.shape))

#Save the 'Id' column
train_ID = train_df['Id']
test_ID = test_df['Id']

#Now drop the  'Id' colum since it's unnecessary for  the prediction process.
train_df.drop("Id", axis = 1, inplace = True)
test_df.drop("Id", axis = 1, inplace = True)

#check again the data size after dropping the 'Id' variable
print("\nThe train data size after dropping Id feature is : {} ".format(train_df.shape)) 
print("The test data size after dropping Id feature is : {} ".format(test_df.shape))

In [ ]:
sns.distplot(train_df["SalePrice"])

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % train_df['SalePrice'].skew())
print("Kurtosis: %f" % train_df['SalePrice'].kurt())

In [ ]:
#correlation matrix
corrmat = train_df.corr()
f, ax = plt.subplots(figsize=(12, 9))
sns.heatmap(corrmat, vmax=.8, square=True);

In [ ]:
#applying log transformation
train_df['SalePrice'] = np.log(train_df['SalePrice'])
sns.distplot(train_df["SalePrice"])

In [ ]:
#data transformation
train_df['GrLivArea'] = np.log(train_df['GrLivArea'])
#transformed histogram and normal probability plot
sns.distplot(train_df['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(train_df['GrLivArea'], plot=plt)

In [ ]:
#data transformation
test_df['GrLivArea'] = np.log(test_df['GrLivArea'])
#transformed histogram and normal probability plot
sns.distplot(test_df['GrLivArea'], fit=norm);
fig = plt.figure()
res = stats.probplot(test_df['GrLivArea'], plot=plt)

In [ ]:
data = [train_df, test_df]
for dataset in data:
    dataset['HasBsmt'] = np.where(dataset['TotalBsmtSF'] > 0, 1, 0)
    dataset['TotalBsmtSF'] = np.where(dataset['HasBsmt'] == 1, np.log(dataset['TotalBsmtSF']), 0)

In [ ]:
#histogram and normal probability plot
sns.distplot(train_df[train_df['TotalBsmtSF']>0]['TotalBsmtSF'], fit=norm);
fig = plt.figure()
res = stats.probplot(train_df[train_df['TotalBsmtSF']>0]['TotalBsmtSF'], plot=plt)

In [ ]:
#Deleting outliers
train_df = train_df.drop(train_df[(train_df['GrLivArea']>4000) & (train_df['SalePrice']<300000)].index)

In [ ]:
    total = train_df.isnull().sum().sort_values(ascending=False)
    percent = (train_df.isnull().sum()/train_df.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    missing_data.head(20)

In [ ]:
#missing data
data = [train_df, test_df]
for dataset in data:
    dataset["PoolQC"] = dataset["PoolQC"].fillna("None")
    dataset["MiscFeature"] = dataset["MiscFeature"].fillna("None")
    dataset["Alley"] = dataset["Alley"].fillna("None")
    dataset["Fence"] = dataset["Fence"].fillna("None")
    dataset["FireplaceQu"] = dataset["FireplaceQu"].fillna("None")
    dataset["LotFrontage"] = dataset.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
    dataset["GarageType"] = dataset["GarageType"].fillna("None")
    dataset["GarageFinish"] = dataset["GarageFinish"].fillna("None")
    dataset["GarageQual"] = dataset["GarageQual"].fillna("None")
    dataset["GarageCond"] = dataset["GarageCond"].fillna("None")
    dataset["GarageYrBlt"] = dataset["GarageYrBlt"].fillna(0)
    dataset["GarageArea"] = dataset["GarageArea"].fillna(0)
    dataset["GarageCars"] = dataset["GarageCars"].fillna(0)
    dataset["BsmtFinSF1"] = dataset["BsmtFinSF1"].fillna(0)
    dataset["BsmtFinSF2"] = dataset["BsmtFinSF2"].fillna(0)
    dataset["BsmtUnfSF"] = dataset["BsmtUnfSF"].fillna(0)
    dataset["TotalBsmtSF"] = dataset["TotalBsmtSF"].fillna(0)
    dataset["BsmtFullBath"] = dataset["BsmtFullBath"].fillna(0)
    dataset["BsmtHalfBath"] = dataset["BsmtHalfBath"].fillna(0)
    dataset["BsmtQual"] = dataset["BsmtQual"].fillna("None")
    dataset["BsmtCond"] = dataset["BsmtCond"].fillna("None")
    dataset["BsmtExposure"] = dataset["BsmtExposure"].fillna("None")
    dataset["BsmtFinType1"] = dataset["BsmtFinType1"].fillna("None")
    dataset["BsmtFinType2"] = dataset["BsmtFinType2"].fillna("None")
    dataset["MasVnrType"] = dataset["MasVnrType"].fillna("None")
    dataset["MasVnrArea"] = dataset["MasVnrArea"].fillna(0)
    dataset['MSZoning'] = dataset['MSZoning'].fillna(dataset['MSZoning'].mode()[0])
    dataset = dataset.drop(['Utilities'], axis=1)
    dataset["Functional"] = dataset["Functional"].fillna("Typ")
    dataset['Electrical'] = dataset['Electrical'].fillna(dataset['Electrical'].mode()[0])
    dataset['KitchenQual'] = dataset['KitchenQual'].fillna(dataset['KitchenQual'].mode()[0])
    dataset['Exterior1st'] = dataset['Exterior1st'].fillna(dataset['Exterior1st'].mode()[0])
    dataset['Exterior2nd'] = dataset['Exterior2nd'].fillna(dataset['Exterior2nd'].mode()[0])
    dataset['SaleType'] = dataset['SaleType'].fillna(dataset['SaleType'].mode()[0])
    dataset['MSSubClass'] = dataset['MSSubClass'].fillna("None") 
    #MSSubClass=The building class
    dataset['MSSubClass'] = dataset['MSSubClass'].apply(str)
    #Changing OverallCond into a categorical variable
    dataset['OverallCond'] = dataset['OverallCond'].astype(str)
    #Year and month sold are transformed into categorical features.
    dataset['YrSold'] = dataset['YrSold'].astype(str)
    dataset['MoSold'] = dataset['MoSold'].astype(str)
    dataset_na = (dataset.isnull().sum() / len(dataset)) * 100
    dataset_na = dataset_na.drop(dataset_na[dataset_na == 0].index).sort_values(ascending=False)
missing_data = pd.DataFrame({'Missing Ratio' :dataset_na})
missing_data.head()

    


In [ ]:
train_df.shape
test_df.shape

In [ ]:
import category_encoders as ce

X = train_df
y = train_df["SalePrice"]
y_train = train_df.SalePrice.values
train_df.drop('SalePrice',axis = 1,inplace=True)

encoder = ce.TargetEncoder(cols=['YrSold', 'MoSold'])
encoder.fit(X, y)
train_df = encoder.transform(X)

X = test_df
test_df = encoder.transform(X,y=None)

#encoder = ce.BinaryEncoder(cols=['FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
#        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
#        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
#        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond']).fit(X, y)
#train_df = encoder.transform(X)
#X = test_df
#test_df = encoder.transform(X)


In [ ]:
y_train

In [ ]:
train_df.shape+test_df.shape

In [ ]:
#new
data = [train_df, test_df]
for dataset in data:
# Adding total sqfootage feature 
    dataset['TotalSF'] = dataset['TotalBsmtSF'] + dataset['1stFlrSF'] + dataset['2ndFlrSF']
    

In [ ]:
from scipy import stats
from scipy.stats import norm, skew #for some statistics

ntrain = train_df.shape[0]
ntest = test_df.shape[0]
all_data = pd.concat((train_df, test_df)).reset_index(drop=True)
#all_data.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(all_data.shape))

numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = all_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    all_data[feat] = boxcox1p(all_data[feat], lam)



In [ ]:
all_data = pd.get_dummies(all_data)
print(all_data.shape)

In [ ]:
train = all_data[:ntrain]
test = all_data[ntrain:]

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

#model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
#                              learning_rate=0.05, n_estimators=720,
#                              max_bin = 55, bagging_fraction = 0.8,
#                              bagging_freq = 5, feature_fraction = 0.2319,
#                              feature_fraction_seed=9, bagging_seed=9,
#                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)

model_lgb = lgb.LGBMRegressor(objective='regression',
                              boosting='dart',
                              num_leaves=5,
                              learning_rate=0.05, 
                              n_estimators=700,
                              max_bin = 55, 
                              bagging_fraction = 0.8,
                              bagging_freq = 5, 
                              feature_fraction = 0.2319,
                              feature_fraction_seed=9, 
                              bagging_seed=9,
                              min_data_in_leaf =6, 
                              min_sum_hessian_in_leaf = 11)

In [ ]:
from sklearn.metrics import mean_squared_error
def rmsle(y, y_pred):
    return np.sqrt(mean_squared_error(y, y_pred))

In [ ]:
model_lgb.fit(train, y_train)
lgb_train_pred = model_lgb.predict(train)
lgb_pred = np.expm1(model_lgb.predict(test.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
import xgboost as xgb
model_xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.1, max_depth=4, 
                             min_child_weight=1.7817, n_estimators=3000,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5, silent=1,
                             random_state =7, nthread = -1)
#subsample=0.5213, nthread = -1, n_estimators=3000

In [ ]:
model_xgb.fit(train, y_train)
xgb_train_pred = model_xgb.predict(train)
xgb_pred = np.expm1(model_xgb.predict(test))
print(rmsle(y_train, xgb_train_pred))

In [ ]:
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
GBoost = GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='huber', max_depth=4,
             max_features='sqrt', max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=15, min_samples_split=10,
             min_weight_fraction_leaf=0.0, n_estimators=3000,
             presort='auto', random_state=None, subsample=1.0, verbose=0,
             warm_start=False)

In [ ]:
GBoost.fit(train, y_train)
gb_train_pred = GBoost.predict(train)
gb_pred = np.expm1(GBoost.predict(test))
print(rmsle(y_train, gb_train_pred))

In [ ]:
xgb_pred

In [ ]:
lgb_pred

In [ ]:
gb_pred

In [ ]:
Final=(lgb_pred + gb_pred + xgb_pred )/3
Price = Final.astype(np.int64)

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = Price
sub.to_csv('submission.csv',index=False)

In [ ]:
sub